# Inport needed package

- import os, sys # to add the parent directory to the path

In [1]:
import os
import sys
import time

- Using torchvision to create a dataset

In [2]:
# Using torchvision to create a dataset
import cv2
from torchvision import transforms
import torch
from torch.utils.data import random_split, DataLoader
import torchvision

import pandas as pd

- import self library

In [3]:
from train.trainer import ClassifierTrainer as Trainer
import dataset as ds  # type: ignore

# Define classification train process

1. Define place where the model is saved

In [4]:
time_str = time.strftime("%Y%m%d_%H%M%S")


2. Define train function

In [5]:
def doTheTrain(dataset, model):
  # define batch_size
  batch_size = 64

  # init train val test ds
  train_val_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_val_size
  train_ds, test_ds = random_split(dataset, [train_val_size, test_size])

  # define optimizer using Adam and loss function
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  trainer = Trainer(model, optimizer, loss_fn, random_seed_value=86)
  print('device: ', trainer.device)
  avg_loss, metric = trainer.cross_validate(train_ds, k=10, epochs=10, batch_size=batch_size)
  print('avg_loss: ', avg_loss)

  # score model
  test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
  model_scored = trainer.score(test_dataloader)
  print(f'model_scored: {model_scored:.4f}, avg_accuracy: {100*(1 - model_scored):.4f}')

  # return model scored, train_avg_lost
  return model_scored, avg_loss

3. execute progress

- define the model

In [ ]:
models = [
    torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
    torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
    torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT),
]

- Define tested datasets

In [7]:
datasets = {
    'gi4e_full': ds.Gi4eDataset(
        './datasets/gi4e',
        transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor()]),
        is_classification=True),
    'gi4e_raw_eyes': ds.ImageDataset(
        './datasets/gi4e_raw_eyes',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
    'gi4e_detected_eyes': ds.ImageDataset(
        './datasets/gi4e_eyes/20250521_200316',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
}

- Train all defined model on each registered datasets

In [8]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for name, dataset in datasets.items():
	for model in models:
		print(f'Running {name} dataset with {model.__class__.__name__}')
		# do the train
		start_time = time.time()
		scored, loss = doTheTrain(dataset, model)
		end_time = time.time()
		total_time = end_time - start_time
		print(f'Finished {name} dataset with {model.__class__.__name__}')
		print('----------------------')

		# save the result
		result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model.__class__.__name__],
			'dataset': [name],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
		})], ignore_index=True)

print('Finished all datasets')

# print the result
print(result_df)



Running gi4e_full dataset with ResNet
device:  cuda
Fold 1/10:
Epoch 1/10, Train Loss: 2.0120, Test Loss: 0.5367, Total Time: 00 hours, 00 minutes, 39 seconds
Epoch 2/10, Train Loss: 0.0091, Test Loss: 0.0030, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 3/10, Train Loss: 0.0008, Test Loss: 0.0005, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 4/10, Train Loss: 0.0003, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 5/10, Train Loss: 0.0002, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 6/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 7/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 8/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 9/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 10/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours,

C:\Users\nnvuf\AppData\Local\Temp\ipykernel_37820\3706550783.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({


Epoch 1/10, Train Loss: 1.5966, Test Loss: 0.8377, Total Time: 00 hours, 00 minutes, 25 seconds
Epoch 2/10, Train Loss: 0.0193, Test Loss: 0.0093, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 3/10, Train Loss: 0.0029, Test Loss: 0.0012, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 4/10, Train Loss: 0.0011, Test Loss: 0.0006, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 5/10, Train Loss: 0.0007, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 6/10, Train Loss: 0.0005, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 7/10, Train Loss: 0.0003, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 8/10, Train Loss: 0.0003, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 9/10, Train Loss: 0.0002, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 10/10, Train Loss: 0.0002, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 17 seconds
Fold 1/10, Total Test Loss: 0.0002, Fol

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

Fold 4/10, Total Test Loss: 0.1589, Fold accuracy: 97.0449
Fold 5/10:
Epoch 1/10, Train Loss: 0.0883, Test Loss: 0.0024, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 2/10, Train Loss: 0.0352, Test Loss: 0.2308, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 3/10, Train Loss: 0.0593, Test Loss: 0.0015, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 4/10, Train Loss: 0.0067, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 5/10, Train Loss: 0.0109, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 6/10, Train Loss: 0.0170, Test Loss: 0.0005, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 7/10, Train Loss: 0.0286, Test Loss: 0.0010, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 8/10, Train Loss: 0.0165, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 9/10, Train Loss: 0.0064, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 10/10, Train Loss: 0.0003, Test Loss: 0.0000, Total Time: 00

...

Fold 1/10, Total Test Loss: 0.1877, Fold accuracy: 81.2306
Fold 2/10:
Epoch 1/10, Train Loss: 0.0939, Test Loss: 1.4121, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 2/10, Train Loss: 0.0637, Test Loss: 0.2989, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 3/10, Train Loss: 0.0061, Test Loss: 0.0030, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 4/10, Train Loss: 0.0004, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 5/10, Train Loss: 0.0002, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 6/10, Train Loss: 0.0001, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 7/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 8/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 9/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 10/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 1/10, Total Test Loss: 0.1345, Fold accuracy: 86.5529
Fold 2/10:
Epoch 1/10, Train Loss: 0.0500, Test Loss: 0.4610, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 2/10, Train Loss: 0.0461, Test Loss: 0.0726, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 3/10, Train Loss: 0.0147, Test Loss: 0.0786, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 4/10, Train Loss: 0.0031, Test Loss: 0.0008, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 5/10, Train Loss: 0.0006, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 6/10, Train Loss: 0.0003, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 7/10, Train Loss: 0.0002, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 8/10, Train Loss: 0.0002, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 9/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 10/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 5/10, Total Test Loss: 0.1666, Fold accuracy: 98.4374
Fold 6/10:
Epoch 1/10, Train Loss: 0.1319, Test Loss: 0.0260, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 2/10, Train Loss: 0.1154, Test Loss: 0.0031, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 3/10, Train Loss: 0.0518, Test Loss: 0.0691, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 4/10, Train Loss: 0.0945, Test Loss: 0.2066, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 5/10, Train Loss: 0.0824, Test Loss: 0.0118, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 6/10, Train Loss: 0.0387, Test Loss: 0.0090, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 7/10, Train Loss: 0.0479, Test Loss: 0.0193, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 8/10, Train Loss: 0.0165, Test Loss: 0.0027, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 9/10, Train Loss: 0.0191, Test Loss: 0.0188, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 10/10, Train Loss: 0.0125, Test Loss: 0.0056, Total Time: 00

4. print the result

In [11]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)

KeyError: "['total_time'] not in index"

In [10]:
print(result_df)

              dataset     model  avg_loss  avg_accuracy
0           gi4e_full    ResNet  0.000008     99.999914
1           gi4e_full  DenseNet  0.000028     99.990581
2           gi4e_full       VGG  0.017698     99.825569
3       gi4e_raw_eyes    ResNet  0.042010     99.709066
4       gi4e_raw_eyes  DenseNet  0.005678     99.373679
5       gi4e_raw_eyes       VGG  0.147762     86.098641
6  gi4e_detected_eyes    ResNet  0.018778     99.837420
7  gi4e_detected_eyes  DenseNet  0.013462     99.948888
8  gi4e_detected_eyes       VGG  0.017937     79.856581
